### Left recursion

In [1]:
def remove_left_recursion(grammar):
    non_terminals = list(grammar.keys())
    
    for A in non_terminals:
        productions = grammar[A]
        new_productions = []
        recursive_productions = []
        
        for production in productions:
            if production.startswith(A):
                recursive_productions.append(production)
            else:
                new_productions.append(production)
        
        if recursive_productions:
            new_A = A + "'"
            new_productions += [p[1:] + new_A for p in recursive_productions]
            new_productions.append('ε')
            grammar[A] = new_productions
            grammar[new_A] = recursive_productions + [new_A]
    
    return grammar

# Example Grammar
grammar = {
    'S': ['T', 'F', 'S+T', 'S-F'],
    'T': ['F', 'T*F'],
    'F': ['(S)', 'id']
}

new_grammar = remove_left_recursion(grammar)
print(new_grammar)


{'S': ['T', 'F', "+TS'", "-FS'", 'ε'], 'T': ['F', "*FT'", 'ε'], 'F': ['(S)', 'id'], "S'": ['S+T', 'S-F', "S'"], "T'": ['T*F', "T'"]}


### Left factoring

In [2]:
def left_factor(grammar):
    factored_grammar = {}
    
    for non_terminal, productions in grammar.items():
        common_prefix = get_common_prefix(productions)
        
        if common_prefix:
            factored_grammar[non_terminal] = [common_prefix + 'X']
            new_productions = [p[len(common_prefix):] or 'ε' for p in productions]
            factored_grammar['X'] = new_productions
        
        else:
            factored_grammar[non_terminal] = productions
    
    return factored_grammar

def get_common_prefix(productions):
    if not productions:
        return None
    
    common_prefix = ''
    min_length = min(len(p) for p in productions)
    
    for i in range(min_length):
        if all(p[i] == productions[0][i] for p in productions):
            common_prefix += productions[0][i]
        else:
            break
    
    return common_prefix

def print_grammar(grammar):
    for non_terminal, productions in grammar.items():
        print(f"{non_terminal} -> {' | '.join(productions)}")

# Example usage
original_grammar = {
    'S': ['abc', 'abd', 'aef', 'aeg'],
    'A': ['abc', 'abd', 'aef', 'aeg'],
}

factored_grammar = left_factor(original_grammar)

print("Original Grammar:")
print_grammar(original_grammar)
print("\nLeft Factored Grammar:")
print_grammar(factored_grammar)


Original Grammar:
S -> abc | abd | aef | aeg
A -> abc | abd | aef | aeg

Left Factored Grammar:
S -> aX
X -> bc | bd | ef | eg
A -> aX


### First and follow 

In [3]:
import sys
import os
sys.setrecursionlimit(60)

def first(string):
    first_ = set()
    if string in non_terminals:
        alternatives = productions_dict[string]

        for alternative in alternatives:
            first_2 = first(alternative)
            first_ = first_ | first_2

    elif string in terminals:
        first_ = {string}

    elif string == '' or string == '@':
        first_ = {'@'}

    else:
        first_2 = first(string[0])
        if '@' in first_2:
            i = 1
            while '@' in first_2:
                first_ = first_ | (first_2 - {'@'})
                if string[i:] in terminals or string[i:] == '':
                    first_ = first_ | {string[i:]}
                    break
                first_2 = first(string[i:])
                first_ = first_ | first_2 - {'@'}
                i += 1
        else:
            first_ = first_ | first_2

    return first_

def follow(nT):
    follow_ = set()
    prods = productions_dict.items()
    if nT == starting_symbol:
        follow_ = follow_ | {'$'}
    for nt, rhs in prods:
        for alt in rhs:
            for i, char in enumerate(alt):
                if char == nT:
                    following_str = alt[i + 1:]
                    if following_str == '':
                        if nt == nT:
                            continue
                        else:
                            follow_ = follow_ | follow(nt)
                    else:
                        follow_2 = first(following_str)
                        if '@' in follow_2:
                            follow_ = follow_ | follow_2 - {'@'}
                            follow_ = follow_ | follow(nt)
                        else:
                            follow_ = follow_ | follow_2

    return follow_

def left_factorize(grammar):
    new_productions = {}
    for non_terminal, alternatives in grammar.items():
        common_prefix = os.path.commonprefix(alternatives)
        if common_prefix:
            suffix_productions = [alt[len(common_prefix):] for alt in alternatives]
            new_non_terminal = non_terminal + "'"
            new_productions[non_terminal] = [common_prefix + new_non_terminal]
            new_productions[new_non_terminal] = suffix_productions

    return new_productions

# Sample Grammar
original_grammar = {
    'S': ['abc', 'abd', 'aef', 'aeg'],
    'A': ['abc', 'abd', 'aef', 'aeg']
}

# Fix input and output
terminals = {'a', 'b', 'c', 'd', 'e', 'f', 'g'}
non_terminals = {'S', 'A'}
starting_symbol = 'S'

productions_dict = original_grammar

# Display Original Grammar
print("Original Grammar:")
for non_terminal, alternatives in original_grammar.items():
    print(f"{non_terminal} -> {' | '.join(alternatives)}")

# Calculate and Display First and Follow Sets
print("\nFirst and Follow Sets:")
for non_terminal in non_terminals:
    first_set = first(non_terminal)
    follow_set = follow(non_terminal)
    print(f"{non_terminal}: First = {first_set}, Follow = {follow_set}")


Original Grammar:
S -> abc | abd | aef | aeg
A -> abc | abd | aef | aeg

First and Follow Sets:
A: First = {'a'}, Follow = set()
S: First = {'a'}, Follow = {'$'}


### Parsing tree

In [4]:
class Node:
    def __init__(self, value):
        self.value = value
        self.children = []

    def add_child(self, child):
        self.children.append(child)

    def __str__(self, level=0):
        result = "  " * level + str(self.value) + "\n"
        for child in self.children:
            result += child.__str__(level + 1)
        return result

def parse_expression(tokens):
    root = Node("Expression")
    current_node = root

    for token in tokens:
        if token.isdigit():
            current_node.add_child(Node("Number: " + token))
        elif token in "+-*/":
            current_node.add_child(Node("Operator: " + token))
        elif token == "(":
            new_node = Node("Expression")
            current_node.add_child(new_node)
            current_node = new_node
        elif token == ")":
            current_node = root  # Move back up one level

    return root

def display_tree(tree):
    print("Parsing Tree:")
    print(tree)

# Example input
expression_tokens = ["(", "3", "+", "5", ")", "*", "2"]

# Parse and display the tree
parsing_tree = parse_expression(expression_tokens)
display_tree(parsing_tree)


Parsing Tree:
Expression
  Expression
    Number: 3
    Operator: +
    Number: 5
  Operator: *
  Number: 2

